In [81]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from typing import TypedDict,Literal,Annotated
from pydantic import BaseModel,Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import operator

In [82]:
generator_llm=ChatOllama(model='gemma3')
evaluator_llm=ChatOllama(model='gemma3')
optimizer_llm=ChatOllama(model='gemma3')

In [83]:
class tweet_state(TypedDict):
    topic:str
    tweet:str
    evaluation:Literal['approved','needs_improvement']
    feedback:str
    iteration:int
    max_iteration:int
    tweet_history:Annotated[list[str],operator.add]
    
    

In [84]:
def generate_tweet(state:tweet_state):
    prompt=[
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]
    result=generator_llm.invoke(prompt).content
    return  {'tweet':result,'tweet_history':[result]}

In [85]:
class eval_schema(BaseModel):
    evaluation:Literal['approved','needs_improvement']=Field(...,description='Final evalution result')
    feedback:str=Field(...,description='Feedback for the tweet')

In [86]:
parser=PydanticOutputParser(pydantic_object=eval_schema)

In [87]:
def evaluate_tweet(state:tweet_state):
    template= ChatPromptTemplate.from_messages(
    [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content="""
Evaluate the following tweet:

Tweet:{tweet}

Use the criteria below to evaluate the tweet:

1. Originality-Is this fresh, or have you seen it a hundred times before?  
2. Humor - Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness - Is it short, sharp, and scroll-stopping?  
4. Virality Potential - Would people retweet or share it?  
5. Format - Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")])
    prompt_with_partial = template.partial(format_instruction=parser.get_format_instructions())
    
    prompt=prompt_with_partial.format(tweet=state['tweet'])
    result=evaluator_llm.invoke(prompt)
    final_result=parser.parse(result.content)
    return {'evaluation':final_result.evaluation,'feedback':final_result.feedback}

    

In [88]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

def evaluate_tweet(state: tweet_state):
    # Step 1: Create prompt template WITHOUT partial_variables
    template = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(
            "You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."
        ),
        HumanMessagePromptTemplate.from_template("""
Evaluate the following tweet:

Tweet: {tweet}

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Don't end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses
{format_instruction}
""")
    ])

    # Step 2: Add partial variable separately
    prompt_with_partial = template.partial(format_instruction=parser.get_format_instructions())

    # Step 3: Format messages with the actual tweet content
    messages = prompt_with_partial.format_messages(tweet=state['tweet'])

    # Step 4: Invoke the model
    result = evaluator_llm.invoke(messages)

    # Step 5: Parse result using your parser
    final_result = parser.parse(result.content)

    return {
        'evaluation': final_result.evaluation,
        'feedback': final_result.feedback
    }


In [89]:
def optimize_tweet(state:tweet_state):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]
    
    result=optimizer_llm.invoke(messages).content
    iteration=state['iteration']+1
    return {'tweet':result,'iteration':iteration}

In [90]:
def route_evaluation(state:tweet_state):
    if state['evaluation']=='approved' or state['iteration']>=state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [91]:
graph=StateGraph(tweet_state)
graph.add_node('generate',generate_tweet)
graph.add_node('evaluate',evaluate_tweet)
graph.add_node('optimize',optimize_tweet)

graph.add_edge(START,'generate')
graph.add_edge('generate','evaluate')

graph.add_conditional_edges('evaluate',route_evaluation,{'approved':END,'needs_improvement':'optimize'})
graph.add_edge('optimize','evaluate')

wf=graph.compile()

In [93]:
initial_state = {
    "topic": "Indian Railways",
    "iteration": 1,
    "max_iteration": 5
}
result=wf.invoke(initial_state)

In [96]:
result

{'topic': 'Indian Railways',
 'tweet': 'Okay, here’s a revised tweet based on your feedback, aiming for a short, viral-worthy piece focusing on a single arresting observation and minimizing cliché:\n\n**Revised Tweet:**\n\n“The rhythmic clatter of the Indian Railways…and a silent, desperate plea for a new passport. My pulse still echoes. 🤯 #IndianRailways #TravelNightmare”\n\n---\n\n**Reasoning for Changes:**\n\n*   **Removed Q&A:** Straight to the point.\n*   **Combined Imagery:** “Rhythmic clatter” immediately establishes the setting and introduces the core anxiety.\n*   **Stronger Emotion:** “Silent, desperate plea” retained but streamlined.\n*   **Emoji Placement:** The exploding head emoji (🤯) feels more integrated into the visceral feeling of the experience, rather than just tacked on.\n*   **Conciseness:**  Kept well under 280 characters.\n*   **Focus on Observation:** The tweet centers around a single, impactful detail - the sound of the trains - to make it more arresting.\n\nH

In [97]:
for tweet in result['tweet_history']:
    print(tweet)
    print("***")
    

Okay, here's a tweet for you:

My therapist told me to embrace the chaos. So I booked a 12-hour overnight train in India. Pretty sure my anxiety just found a new home. 🚂💨 #IndianRailways #TravelFail #SendHelp
***
